<a href="https://colab.research.google.com/github/hanghae-plus-AI/AI-1-JunYear/blob/main/week%205/%5B5%EC%A3%BC%EC%B0%A8%5D_%EA%B8%B0%EB%B3%B8%EA%B3%BC%EC%A0%9C_%EB%89%B4%EC%8A%A4_%EA%B8%B0%EC%82%AC_%EB%B6%84%EB%A5%98%EB%A5%BC_Gemma%EB%A1%9C_%EA%B5%AC%ED%98%84%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [5주차 기본과제] 뉴스 기사 분류를 Gemma로 구현하기

이번 과제에서는 이전 주차 과제에서 활용했던 `fancyzhx/ag_news` 문제를 zero-shot classification으로 푸시면 됩니다. 아래 사항들에 유의하시면 될 것 같습니다.

- [ ]  Label들을 올바르게 text화 하여 넘겨주셔야 합니다.
- [ ]  `test` split data 50개에 대한 정확도 계산 코드 및 출력이 남아있어야 합니다.

이외에는 Gemma-2B 모델의 logit 계산 능력을 활용한다는 부분 빼고는 제약이 없습니다.

In [1]:
!pip install datasets

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.7 MB/s eta 0:00:00


그 다음 Gemma-2B를 사용하기 위해 다음과 같은 작업을 진행합니다:
1. huggingface.co 계정 만들고 로그인하기
2. https://www.kaggle.com/models/google/gemma/license/consent 에서 Gemma license 동의하기
3. 홈 화면으로 돌아와, `Profile > Settings > Access Tokens` 메뉴로 들어와 "Write" type의 token 생성하기
4. 생성한 토큰을 아래 "HF TOKEN"에 불여넣고 셀을 실행하기.

In [6]:
from huggingface_hub import login
from google.colab import userdata


login(userdata.get('HF_TOKEN'))

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


정상적으로 token을 생성하고 Gemma license에 동의했다면 아래 코드로 tokenizer와 Gemma-2B 모델을 불러올 수 있습니다.

In [8]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b")
model = AutoModelForCausalLM.from_pretrained("google/gemma-2b", device_map="auto")

tokenizer_config.json:   0%|          | 0.00/33.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [9]:
model

GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-17): 18 x GemmaDecoderLayer(
        (self_attn): GemmaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): GemmaRotaryEmbedding()
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (up_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (down_proj): Linear(in_features=16384, out_features=2048, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): GemmaRMSNorm((2048,), eps=1e-06)
        (post_attention_layernorm): GemmaRMSNorm((2048,), eps=1e-

In [10]:
import torch

def zero_shot_classification(text, task_description, labels):  # text는 주어진 입력, task_description은 task에 대한 설명, labels은 class들을 text로 변환한 결과입니다.
    text_ids = tokenizer(task_description + text, return_tensors="pt").to("cuda")  # 먼저 task_description과 text를 이어붙인 후, tokenize합니다.
    probs = []
    for label in labels:  # 그 다음 각 text화된 label들을 tokenize하고 입력에 이어붙인 후, Gemma-2B에 넣어줍니다.
        label_ids = tokenizer(label, return_tensors="pt").to("cuda")
        n_label_tokens = label_ids['input_ids'].shape[-1] - 1  # text로 변환한 label의 token 수를 계산합니다.
        input_ids = {
            'input_ids': torch.concatenate([text_ids['input_ids'], label_ids['input_ids'][:, 1:]], axis=-1),  # concatenate 명령어를 통해 이어붙이는 모습입니다.
            'attention_mask': torch.concatenate([text_ids['attention_mask'], label_ids['attention_mask'][:, 1:]], axis=-1)
        }

        logits = model(**input_ids).logits  # Logit을 계산한 모습입니다.
        prob = 0
        n_total = input_ids['input_ids'].shape[-1]
        for i in range(n_label_tokens, 0, -1):  # 일반적으로 text로 변환한 label은 여러 token으로 이루어져있습니다. 이러한 label에 대한 logit은 구성하는 모든 token들의 logit들의 합으로 정의합니다.
            token = label_ids['input_ids'][0, i].item()
            prob += logits[0, n_total - i, token].item()
        probs.append(prob)

        del input_ids
        del logits
        torch.cuda.empty_cache()  # 위의 del과 empty_cache() 명령어를 통해 GPU를 제때 할당해제 해줍니다. 만약 GPU가 여유롭다면 지워주시는게 속도적으로 이득입니다.

    return probs

아래는 실제로 zero-shot classification을 해본 결과입니다.

In [11]:
probs = zero_shot_classification("I am happy!", "Is the sentence positive or negative?: ", ["positive", "negative"])
print(probs)

[-4.515157699584961, -9.59005355834961]


보시다시피 우리는 Gemma를 별도로 학습하지 않았음에도 불구하고 주어진 문장이 긍정적이라는 것을 정확하게 예측하고 있습니다.

다음은 영화 리뷰 감정 분석 task에 적용해봅시다.
먼저 data를 불러옵니다.

In [12]:
from datasets import load_dataset

# 데이터셋 imdb -> ag_news
ag_news = load_dataset("fancyzhx/ag_news")

# label (train) text
label_names = ag_news['train'].features['label'].names
print(label_names)

def preprocess_function(examples):
    return tokenizer(examples["text"], max_length=200, truncation=True)

tokenized_ag_news = ag_news.map(preprocess_function, batched=True)

README.md:   0%|          | 0.00/8.07k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

['World', 'Sports', 'Business', 'Sci/Tech']


Map:   0%|          | 0/120000 [00:00<?, ? examples/s]

Map:   0%|          | 0/7600 [00:00<?, ? examples/s]

In [13]:
ag_news["train"][0]

{'text': "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.",
 'label': 2}

In [14]:
tokenized_ag_news

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 120000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 7600
    })
})

그리고 `test` data에서 50개의 영화 리뷰에 대해 예측하는 코드는 다음과 같습니다.

In [20]:
import numpy as np
from tqdm import tqdm


n_corrects = 0
n_total = 50

label_features = ag_news['train'].features['label'].names

for i in tqdm(range(n_total)):
    text = tokenized_ag_news['test'][i]['text']
    label = tokenized_ag_news['test'][i]['label']
    probs = zero_shot_classification(
        text,
        "What is Topic of this article?: ",
        #labels=["World", "Sports", "Business", "Sci/Tech"]
        labels=label_features
    )

    pred = np.argmax(np.array(probs))
    if pred == label:
        n_corrects += 1

acc = n_corrects / n_total

print()
print(f"Total Correctness: {n_corrects}개")
print(f"Accuracy: {acc}")

100%|██████████| 50/50 [00:10<00:00,  4.82it/s]


Total Correctness: 35개
Accuracy: 0.7
